<a href="https://colab.research.google.com/github/Gwonchankim/AI-python-connect/blob/master/n122-chi-square-test/n122a-chi-square-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 2 / NOTE 2*

# 📝 Assignment

# 카이제곱검정


## 1. 공공데이터 조작 연습. 

국가 통계 포털에서 아래 이미지를 참조하여

<https://kosis.kr/statisticsList/statisticsListIndex.do?menuId=M_01_01&vwcd=MT_ZTITLE&parmTabId=M_01_01#SelectStatsBoxDiv>

**2020년 8월**에 해당하는 규모별 미분양현황 에 대한 데이터셋을 생성하세요. 

<img src='https://i.imgur.com/aZc4UJO.png' width = 500>

- `60m이하`, `60~85m`, `85m초과`의 3개 규모와 (column)
- `서울`, `대전`, `대구`, `부산` 4개의 지역을 포함해야합니다. (row)
- `민간부문`만 포함합니다
- 데이터가 없는 경우는 0으로 처리하세요.

이후 데이터셋을 colab으로 불러오세요. 이때 변수의 이름은 `df`를 사용합니다.


In [4]:
import pandas as pd
import numpy as np
from scipy import stats

In [6]:
df = pd.read_csv('/content/규모별_미분양현황_20210716162818.csv', encoding='euc-kr')
df.columns = ['시도', '부문', '규모', '2020_08']

df= df.pivot_table ( index = '시도' ,  columns = '규모' ,  values = '2020_08', aggfunc='sum')
df = df.rename(columns = {'60㎡이하' : '~60㎡', '85㎡초과' :'85㎡~'})
df[['60∼85㎡',	'~60㎡',	'85㎡~']] = df[['60∼85㎡',	'~60㎡',	'85㎡~']].apply(pd.to_numeric)
col0, col1, col2 =[df.columns[0]] , [df.columns[1]] , [df.columns[2]]
new_col=col1+col0+col2
df=df[new_col]
df

규모,~60㎡,60∼85㎡,85㎡~
시도,,,
대구,143,1437,44
대전,782,1,0
부산,577,735,142
서울,54,2,0


In [7]:
df.shape

(4, 3)

In [8]:
df.sum().sum()

3917

## 2. 지역에 대해서 one-sample chi-square test를 실행, 해당 결과를 `chi1`에 저장 후 설명해보세요.

예시) 만약 **9월달 데이터**를 기준으로 한다면
```python
 [52+2+0, 590+665+142, 113+1061+42, 772+1+0]
```
을 비교 하게 될 것입니다.




In [10]:
# scipy 사용
from scipy.stats import chisquare
obs = df.sum(axis=1)
chi1 = chisquare(obs)[0]
print(f'chi1 = {chi1:}')

chi1 = 1564.4572376818994


In [14]:
obs

시도
대구    1624
대전     783
부산    1454
서울      56
dtype: int64

## Ho : 지역에 따라 미분양 수는 독립적이다.
    

*   지역에 상관없이 공평하게 나올 것이다.
*   p-value = 0 < 0.05, 기각


## H1 : 지역에 따라 미분양 수는 독립적이지 않다.

*   지역에 상관없이 공평하게 나오지 않을 것이다.
*   p-value = 0 < 0.05, 채택


## 결론 : 지역변 미분양 수는 지역과 연관성이 있을것이다.


## 3. 지역과 규모에 대해서 two-sample chi-square test를 실행, 해당 결과를 `chi2`에 저장 후 설명해보세요.

예시) **9월달 데이터**를 기준으로 한다면

| |-60 | 60-85 | 85- |
|:-:|:-:|:-:|:-:|
|서울|52|2|0|
|대전|772|1|0|
|대구|113|1061|42|
|부산|590|665|142|

에 대해서 검정해야 할 겁니다.




In [15]:
from scipy.stats import chi2_contingency
chi2 = chi2_contingency(df) #df -> 이미 crosstab이 되어있네.
chi2
print(f' chi2 = {chi2[0]}\n p_value = {chi2[1]}\n 자유도 = {chi2[2]}\n array = \n{chi2[3]}')

 chi2 = 2064.5767314171994
 p_value = 0.0
 자유도 = 6
 array = 
[[645.12228746 901.76155221  77.11616033]
 [311.04110288 434.77789124  37.18100587]
 [577.59101353 807.36533061  69.04365586]
 [ 22.24559612  31.09522594   2.65917794]]


## H0 : 미분양 현황에 대해 지역과 규모는 연관성이 없다
## H1 : 미분양 현황에 대해 지역과 규모는 연관성이 있다

      * 검정결과, p-value는 0.0으로 유의수준 95%하에서 유의확률 0.05보다 작은
        값이다. 따라서 귀무가설을 기각한다. 
        즉, 미분양 현황에 있어 지역과 규모는 연관성이 없다고 볼 수 없다.`

## 4. 2번에 대해서 NumPy 를 사용하여 (Scipy를 사용하지 않고) $\chi^2$ test 시행 후 2번의 결과와 비교해보세요. 

- `obs`, `exp`, `chi`라는 변수를 사용해야합니다.

In [17]:
# numpy사용
obs = df.sum(axis = 1)
exp = np.sum(obs) / len(obs)
chi_squared = ((obs - exp)**2) / exp
chi = chi_squared.sum()
p_value = 1 - stats.chi.cdf(chi, df = (4-1))
print(f'chi = {chi:}' ,f'p_value = {p_value:}')

chi = 1564.4572376818994 p_value = 0.0


In [18]:
exp

979.25

## 🔥 도전 과제

아래 세가지를 다하면 됩니다.

### 1. Function

4번에서 사용한 one sample chisquare test를 함수의 형태로 변경하세요.

In [ ]:
v1 = [18,22,20,15,23,22]
v2 = [5,23,26,19,24,23]

def myChisq(value):
  import numpy as np
  from scipy import stats
  obs = value                   # v1, v2
  exp = np.sum(obs) / len(obs)  # 기대값
  chi_squared = ((obs-exp)**2) / exp     # x제곱
  chi = chi_squared.sum()                # chisqure
  p_value = 1 - stats.chi2.cdf(chi, df = len(value)-1)  # p_value
  return print(f'value = {value}, chisquare = {chi} , p_value = {p_value}') 

myChisq(v1) 
myChisq(v2)



value = [18, 22, 20, 15, 23, 22], chisquare = 2.3000000000000003 , p_value = 0.8062668698851285
value = [5, 23, 26, 19, 24, 23], chisquare = 14.8 , p_value = 0.011251979028327308


### 2. ANOVA

아래 링크를 참조하여 ANOVA 에 대한 글을 읽고

<https://partrita.github.io/posts/ANOVA-python/>

다음 `4개 그룹에 대해서 평균의 차이가 있는지`에 대한 가설 검정을 시행하세요.

A : `38 33 35 92 76 97 88 41 11  9`

B : `18 52 62 48 30 40 87 12 97 82`

C :  `28 90  5 49 66 73 96 80  4 17`

D : ` 8 99  4 12  7 64 18 10  9 20`



## H0 : 4개의 그룹에 대해서 평균의 차의가 나지 않는다
## H1 : 4개의 그룹에 대해서 평균의 차이가 난다.

In [ ]:
A = [38,33,35,92,76,97,88,41,11,9]
B = [18,52,62,48,30,40,87,12,97,82]
C = [28,90,5,49,66,73,96,80,4,17]
D = [8,99,4,12,7,64,18,10,9,20]

F_statistic, pVal = stats.f_oneway(A,B,C,D)
print(f'데이터의 일원분산분석 결과 : F={F_statistic}, p={pVal}')

데이터의 일원분산분석 결과 : F=1.7249594239128412, p=0.17920877113948797


## 결과
*    P-value>0.05 이므로 귀무가설 기각 대립가설 채택